In [1]:
using DifferentiableFactorizations, LinearAlgebra, ComponentArrays

In [2]:
A = rand(5, 3)

5×3 Matrix{Float64}:
 0.956663  0.709191  0.997825
 0.162243  0.480874  0.162824
 0.298358  0.518527  0.910157
 0.631994  0.264437  0.421783
 0.342308  0.612628  0.586

In [3]:
x = diff_qr(A)
(; Q, R) = x

(Q = [-0.7691211696761429 0.14227750042512496 -0.006945868722774057; -0.13043761284976144 -0.563934959784967 0.6107103529403375; … ; -0.508099221657185 0.4267890986967863 0.15597740732173246; -0.2752030824270174 -0.5343306124780337 0.1115876299570795], R = [-1.2438394127769836 -1.035513742339441 -1.3825816849669903; 0.0 -0.6131991238279286 -0.48392064885984254; 0.0 0.0 -0.4755391870263819])

In [4]:
@time begin
    vcat(
        vec(UpperTriangular(Q' * Q) + LowerTriangular(R) - I - Diagonal(R)),
        vec(Q * R - A),
    )
end

  2.311390 seconds (5.85 M allocations: 266.937 MiB, 6.47% gc time, 99.81% compilation time)


24-element Vector{Float64}:
  0.0
  0.0
  0.0
 -7.085092828162905e-17
 -2.220446049250313e-16
  0.0
  4.6449410103673015e-17
  6.96194973671492e-17
  4.996003610813204e-16
  1.1102230246251565e-16
  ⋮
 -1.1102230246251565e-16
  0.0
  1.1102230246251565e-16
  0.0
  1.1102230246251565e-16
 -1.6653345369377348e-16
  1.1102230246251565e-16
  5.551115123125783e-17
 -1.1102230246251565e-16

In [5]:
@time begin
    m, n = size(A, 1), size(A, 2)
    res_A = copy(A)
    res_R = Matrix{eltype(A)}(I, n, n)
    for i in 1:m
      for j in 1:n
        for k in 1:n
          res_A[i, j] -= Q[i, k] * R[k, j]
          if i == 1 && j < k                                                                                                     
            res_R[k, j] += R[k, j]
          end
          if j <= k
            res_R[j, k] -= Q[i, k] * Q[i, j]
          end
        end
      end
    end
    vcat(vec(res_A), vec(res_R))
end

  0.023781 seconds (2.46 k allocations: 131.417 KiB, 98.84% compilation time)


24-element Vector{Float64}:
 -1.1102230246251565e-16
  0.0
 -5.551115123125783e-17
  0.0
  0.0
 -1.5265566588595902e-16
  1.1102230246251565e-16
 -5.551115123125783e-17
 -1.1102230246251565e-16
  0.0
  ⋮
  2.7755575615628914e-17
  0.0
  0.0
  8.326672684688674e-17
  2.220446049250313e-16
  0.0
 -4.5102810375396984e-17
 -8.326672684688674e-17
 -4.961309141293668e-16